In [4]:
import pandas as pd
import os
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import albumentations as A

In [5]:
df = pd.read_csv("../Dataset.csv")
df.head(3)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link,ratings,review
0,15970.jpg,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"High quality fabric, very satisfied. for every..."
1,39386.jpg,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Exceeded expectations, love the fit! and I'm v..."
2,59263.jpg,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy. Would rec..."


In [ ]:
transform = A.Compose([
    A.GaussianBlur(blur_limit=(3, 7), p=0.7), 
    A.Rotate(limit=(-45, 45), p=0.7),
    A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.7),  
    # A.HorizontalFlip(p=0.7), 
    # A.VerticalFlip(p=0.5),  
    # A.RandomResizedCrop(height=256, width=256, scale=(0.8, 1.0), p=0.7), 
    # A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2, p=0.6), 
])

num_augmented_images = 3

# augmented_image_links = []

In [ ]:
os.makedirs("augmented_images", exist_ok=True)

In [ ]:
for index, row in df.head(100).iterrows():  # Change here to only get the first 100 images
    image_url = row['link']
    image_id = row['id']

    # Create a subdirectory for the current image id
    id_folder = os.path.join("augmented_images", image_id)
    os.makedirs(id_folder, exist_ok=True)

    try:
        # Fetch the original image from the URL
        response = requests.get(image_url)
        original_img = Image.open(BytesIO(response.content)).convert("RGB")

        # Save the original image locally
        original_img_path = os.path.join(id_folder, f"{image_id}_original.jpg")
        original_img.save(original_img_path, format='JPEG')

        # Convert the image to a numpy array for augmentation
        img_np = np.array(original_img)

        # Generate multiple augmented images
        for i in range(num_augmented_images):
            augmented = transform(image=img_np)
            augmented_img = Image.fromarray(augmented['image'])
            
            # Define a unique filename for the augmented image
            augmented_filename = f"{image_id}_augmented_{i + 1}.jpg"
            augmented_img_path = os.path.join(id_folder, augmented_filename)

            # Save the augmented image locally
            augmented_img.save(augmented_img_path, format='JPEG')

    except Exception as e:
        print(f"Failed to process {image_id}: {e}")

print("Image augmentation for the first 10 images completed and saved to the 'augmented_images' directory!")

Image augmentation for the first 10 images completed and saved to the 'augmented_images' directory!
